расчет coherence базируется на двух типов данных
(бибилиотека из gensim)
1. датасет
2. тематическое решение

In [1]:
# читаем датасет
import numpy as np
import pandas as pd
# load dataset from file csv

# level1 - lenta
#f = 'C:/koltcov_python_scripts/Word_embeding project/lenta/level1/ETM_lenta1_data/lenta_lem_1_coherence.csv'

# level2 - lenta
#f = 'C:/koltcov_python_scripts/Word_embeding project/lenta/level2/data/lenta_lem_2_utf8_coherence.csv'

# level2 - 20topicsnews
#f = 'C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level2/20topicsnews_prep_2/20topicsnews_lem_2_analysis.csv'

# level1 - 20topicsnews
f = 'C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/20topicsnews_prep_1/20topicsnews_lem_1_analysis.csv'


df=pd.read_csv(f,  sep=';', encoding='utf8')
#df=pd.read_csv(f,  sep=';', encoding='cp1252' )
#df=pd.read_csv(f,  sep=';', encoding='Latin-1')
# df=pd.read_csv(f,  sep=';', encoding='ANSI')

# this gives us the size of the array
print('Dataset size', df.shape)

# here we can get size of array as two variables
num_rows, num_feature = df.shape

print('row number: ', num_rows)
print('feature number: ', num_feature)
print()
print('names of features: ', list(df))

print('-------------------')
print('full data loaded')
print('-------------------')

Dataset size (15425, 1)
row number:  15425
feature number:  1

names of features:  ['docs']
-------------------
full data loaded
-------------------


In [2]:
import nltk
# download Tokenizer 
nltk.download("punkt")
# создание объекта nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import time
import re
# there will be a list of lematized texts
y_data = []
start = time.time()
# помним, что датасет у нас уже лематизирован.
# конвертируем датасет в список
y_data=df['docs'].to_list()
stop = time.time()    
# results is set lematized documents
print('time of execution (sec)', stop - start)

time of execution (sec) 0.0010085105895996094


In [4]:
import gensim
import re
import time

from gensim import corpora
start = time.time()
# Tokenize(split) the sentences into words
texts = [[text for text in doc.split()] for doc in y_data]
# Create dictionary
dictionary = corpora.Dictionary(texts)
# create a corpus
gensim_dictionary = corpora.Dictionary(texts)
gensim_corpus = [gensim_dictionary.doc2bow(token, allow_update=True) for token in texts]

# Get information about the dictionary and corpus
print(gensim_dictionary)
print('number of tokens: ', len(gensim_dictionary))
print('number of docs: ', len(gensim_corpus))
stop = time.time()    
# results is set lematized documents
print('time of execution (sec)', stop - start)

Dictionary(41200 unique tokens: ['actual', 'article', 'available', 'chong', 'frampton']...)
number of tokens:  41200
number of docs:  15425
time of execution (sec) 9.655490159988403


In [6]:
#print(gensim_dictionary.token2id)
#print()
#print(gensim_dictionary[1122])

In [9]:
# читаем список файлов в каталоге
# указываем каталог, в котором лежат матрицы PHI
results0 = 'C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/'
# указываем каталог, в который будем склдаывать результаты выдления матрицы phi
results1 = 'C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/'
results2 = 'coherence_wiki-news-300d-1M_PHI_0_lev1_run2.csv'

In [10]:
import os
import pickle
import pandas as pd
import numpy as np
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel

all_coherence = []
topics_num = []

# организуем цикл по этим файлам
for i in os.listdir(results0):
    print('file name: ', results0+i)
    f = results0+i
    phidf = pd.read_csv(f,  sep=';', encoding='utf8')
    print('Dataset size', phidf.shape)
    num_rows, num_feature = phidf.shape
    #print('row number: ', num_rows)
    #print('feature number: ', num_feature)
    phidf=phidf.rename(columns={'Unnamed: 0': 'words'})
    mytopics = []
    # создаем список тем столбцов
    topics_name =list(phidf)
    #topics_name.remove('words')
    # цикл по именам столбцов
    for i in topics_name:
        # сортируем по текущему столбцу
        sorted_df = phidf.sort_values(by=i, ascending = False)
        mytopics.append(sorted_df['words'].head(5).to_list())
    print('list of topics: ', mytopics)
    print('number of topics: ', len(mytopics))
    # вычисление coherence
    cm = CoherenceModel(topics=mytopics, corpus=gensim_corpus, dictionary=gensim_dictionary, coherence='u_mass')
    coherence = cm.get_coherence()  # get coherence value
    print(coherence)
    all_coherence.append(coherence)
    topics_num.append(len(mytopics))
    print('')
    

import csv
from itertools import zip_longest
d = [topics_num, all_coherence]
export_data = zip_longest(*d, fillvalue = '')

with open(results1 + results2, 'w', encoding="utf", newline='') as myfile:
      wr = csv.writer(myfile,  delimiter=';')
      wr.writerow(("Number of clusters","Coherence"))
      wr.writerows(export_data)
myfile.close()

file name:  C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/wiki-news-300d-1M_10_train_0_5_Phi.csv
Dataset size (12110, 11)
list of topics:  [['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'], ['have', 'thoma', 'instal', 'jame', 'charle'], ['christian', 'armenian', 'israel', 'religion', 'turkish'], ['some', 'people', 'other', 'more', 'than'], ['they', 'doesn', 'know', 'century', 'unless'], ['make', 'know', 'like', 'want', 'think'], ['window', 'file', 'computer', 'mail', 'card'], ['david', 'netcom', 'john', 'paul', 'steve'], ['they', 'century', 'koresh', 'start', 'allude'], ['they', 'complicate', 'allude', 'hasn', 'rayssd'], ['datum', 'tumor', 'upenn', 'keysym', 'function']]
number of topics:  11
-5.513613468351234

file name:  C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/wiki-news-300d-1M_11_train_0_52_Phi.csv
Dataset size (12110, 12)
list of topics:  [['zyeh', 'zuri

-5.071276072618402

file name:  C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/wiki-news-300d-1M_19_train_0_109_Phi.csv
Dataset size (12110, 20)
list of topics:  [['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'], ['some', 'more', 'other', 'than', 'like'], ['uiuc', 'spacecraft', 'solar', 'lunar', 'moon'], ['article', 'they', 'talk', 'page', 'jame'], ['faith', 'jesus', 'scripture', 'religion', 'bible'], ['people', 'know', 'come', 'anyone', 'please'], ['window', 'file', 'system', 'mail', 'computer'], ['have', 'armenian', 'arab', 'turkish', 'israeli'], ['every', 'next', 'whenever', 'whichever', 'ahead'], ['optilink', 'paramax', 'gibbon', 'idol', 'amount'], ['have', 'david', 'john', 'michael', 'steve'], ['kill', 'hear', 'child', 'fire', 'burn'], ['format', 'input', 'function', 'generate', 'write'], ['datum', 'netcom', 'koresh', 'digex', 'instal'], ['when', 'which', 'because', 'year', 'then'], ['drive', 'chip', 'sell', 'game', 'play

list of topics:  [['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'], ['like', 'think', 'make', 'take', 'look'], ['just', 'only', 'really', 'find', 'here'], ['armenian', 'arab', 'turkish', 'israeli', 'israel'], ['their', 'over', 'every', 'under', 'without'], ['chip', 'drive', 'game', 'price', 'card'], ['file', 'window', 'format', 'function', 'code'], ['uunet', 'politic', 'optilink', 'gatech', 'charle'], ['time', 'year', 'first', 'before', 'after'], ['know', 'right', 'problem', 'thing', 'point'], ['which', 'when', 'because', 'into', 'where'], ['system', 'mail', 'password', 'privacy', 'citizen'], ['medical', 'disease', 'drug', 'medicine', 'chronic'], ['have', 'batf', 'brake', 'baerga', 'generator'], ['come', 'call', 'back', 'kill', 'down'], ['clipper', 'instal', 'usenet', 'microsoft', 'uucp'], ['some', 'other', 'more', 'these', 'than'], ['well', 'good', 'very', 'long', 'high'], ['they', 'edusubject', 'davidian', 'argic', 'iastate'], ['david', 'john', 'steve', 'american', 'michael'], ['artic

-13.664449546631138

file name:  C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/wiki-news-300d-1M_30_train_0_63_Phi.csv
Dataset size (12110, 31)
list of topics:  [['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'], ['know', 'think', 'like', 'thing', 'anyone'], ['have', 'they', 'koresh', 'roby', 'melkonian'], ['hear', 'foot', 'keep', 'morning', 'enter'], ['space', 'nasa', 'satellite', 'science', 'orbit'], ['uiuc', 'netcom', 'unix', 'microsoft', 'scsi'], ['some', 'other', 'more', 'than', 'these'], ['well', 'good', 'very', 'long', 'high'], ['make', 'take', 'look', 'come', 'need'], ['into', 'over', 'back', 'down', 'under'], ['david', 'john', 'american', 'michael', 'steve'], ['right', 'wrong', 'free', 'mark', 'easy'], ['mail', 'info', 'email', 'include', 'list'], ['problem', 'question', 'point', 'fact', 'opinion'], ['uchicago', 'utexas', 'informatik', 'larc', 'higgin'], ['your', 'people', 'their', 'little', 'real'], ['file', 'window'

-4.114617756817945

file name:  C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/wiki-news-300d-1M_35_train_0_62_Phi.csv
Dataset size (12110, 36)
list of topics:  [['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'], ['david', 'john', 'american', 'nasa', 'steve'], ['come', 'call', 'help', 'back', 'drive'], ['post', 'mail', 'number', 'name', 'word'], ['right', 'wrong', 'netcom', 'easy', 'free'], ['time', 'year', 'same', 'every', 'course'], ['iupui', 'patriotbefore', 'polonius', 'phillie', 'banish'], ['jesus', 'christian', 'christ', 'religion', 'love'], ['know', 'think', 'thing', 'question', 'point'], ['iupui', 'patriotbefore', 'banish', 'phillie', 'pervert'], ['problem', 'opinion', 'idea', 'issue', 'report'], ['disease', 'medicine', 'infection', 'cancer', 'medical'], ['system', 'chip', 'card', 'computer', 'phone'], ['datum', 'uiuc', 'uunet', 'scsi', 'gatech'], ['iupui', 'banish', 'patriotbefore', 'pervert', 'msfc'], ['into', 'over',

-4.402858262413484

file name:  C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/wiki-news-300d-1M_39_train_0_181_Phi.csv
Dataset size (12110, 40)
list of topics:  [['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'], ['hear', 'live', 'woman', 'girl', 'wife'], ['nasa', 'apple', 'unix', 'uucp', 'microsoft'], ['file', 'window', 'function', 'code', 'version'], ['know', 'well', 'tell', 'sure', 'understand'], ['very', 'much', 'long', 'high', 'little'], ['your', 'good', 'great', 'strong', 'nice'], ['which', 'also', 'still', 'could', 'must'], ['time', 'course', 'least', 'cost', 'care'], ['space', 'science', 'satellite', 'university', 'mission'], ['david', 'john', 'american', 'steve', 'michael'], ['include', 'datum', 'follow', 'provide', 'available'], ['such', 'even', 'without', 'consider', 'though'], ['just', 'thing', 'think', 'something', 'really'], ['onto', 'whereas', 'unsigned', 'whichever', 'previous'], ['same', 'number', 'every', 'di

-6.297476099781401

file name:  C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/wiki-news-300d-1M_43_train_0_110_Phi.csv
Dataset size (12110, 44)
list of topics:  [['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'], ['netherland', 'hoene', 'wisc', 'membersof', 'multitude'], ['email', 'info', 'mail', 'information', 'user'], ['problem', 'thing', 'something', 'anything', 'wrong'], ['which', 'then', 'could', 'should', 'must'], ['christian', 'child', 'jesus', 'world', 'religion'], ['question', 'point', 'post', 'fact', 'opinion'], ['article', 'uunet', 'taxis', 'xview', 'livesey'], ['netherland', 'hoene', 'wisc', 'membersof', 'multitude'], ['unix', 'uucp', 'koresh', 'scsi', 'xterm'], ['drive', 'chip', 'game', 'card', 'play'], ['david', 'uiuc', 'john', 'american', 'clinton'], ['people', 'anyone', 'please', 'someone', 'didn'], ['file', 'window', 'program', 'image', 'graphic'], ['netherland', 'wisc', 'hoene', 'membersof', 'louray'], ['well

-4.991116213886928

file name:  C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/wiki-news-300d-1M_47_train_0_164_Phi.csv
Dataset size (12110, 48)
list of topics:  [['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'], ['american', 'john', 'clinton', 'washington', 'april'], ['deviation', 'thegospel', 'suggest', 'hewa', 'viamar'], ['state', 'public', 'federal', 'university', 'government'], ['well', 'good', 'very', 'long', 'high'], ['people', 'anyone', 'someone', 'person', 'woman'], ['only', 'here', 'still', 'exist', 'however'], ['system', 'program', 'drive', 'chip', 'computer'], ['christian', 'jesus', 'christ', 'bible', 'love'], ['kill', 'child', 'fire', 'live', 'crime'], ['some', 'other', 'these', 'many', 'those'], ['which', 'also', 'doesn', 'didn', 'often'], ['right', 'point', 'line', 'make', 'wrong'], ['uiuc', 'callison', 'lehigh', 'lonestar', 'tulsa'], ['speed', 'sell', 'price', 'electric', 'gear'], ['your', 'thoma', 'livesey', '

-8.878384982640899

file name:  C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/wiki-news-300d-1M_5_train_0_144_Phi.csv
Dataset size (12110, 6)
list of topics:  [['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'], ['they', 'bosio', 'nobody', 'reston', 'uiuc'], ['have', 'netcom', 'jame', 'koresh', 'charle'], ['file', 'datum', 'window', 'format', 'function'], ['they', 'lookingfor', 'uncover', 'week', 'hobokenite'], ['some', 'know', 'like', 'make', 'other']]
number of topics:  6
-8.03603102314041

file name:  C:/koltcov_python_scripts/Word_embeding project/20topicsnews/level1/level1/run2/0_add/wiki-news-300d-1M_PHI/wiki-news-300d-1M_6_train_0_165_Phi.csv
Dataset size (12110, 7)
list of topics:  [['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'], ['file', 'code', 'email', 'download', 'copy'], ['your', 'other', 'some', 'more', 'like'], ['have', 'they', 'uiuc', 'uunet', 'jame'], ['article', 'know', 'think', 'want', 'thing'], ['person', '

In [9]:
mytopics

[['zyeh', 'zurich', 'zuma', 'zrepachol', 'zoom'],
 ['article', 'problem', 'write', 'read', 'thank'],
 ['christianity', 'bible', 'point', 'mean', 'christian'],
 ['know', 'some', 'people', 'just', 'like'],
 ['nasa', 'research', 'space', 'science', 'satellite'],
 ['jame', 'doug', 'kevin', 'davidian', 'uiuc'],
 ['well', 'good', 'they', 'work', 'right'],
 ['have', 'your', 'when', 'their', 'time'],
 ['file', 'window', 'system', 'program', 'software'],
 ['state', 'turkish', 'israeli', 'armenian', 'federal']]

In [46]:
# читаем матрицу слов как датафрейм - тематическое решение
f = 'C:/koltcov_python_scripts/Word_embeding project/lenta/level1/run1/0/300_wiki_embeddings_PHI/300_wiki_embeddings_22_train_0_187_Phi.csv'
phidf=pd.read_csv(f,  sep=';', encoding='ANSI')
# this gives us the size of the array
print('Dataset size', phidf.shape)

# here we can get size of array as two variables
num_rows, num_feature = phidf.shape

print('row number: ', num_rows)
print('feature number: ', num_feature)
print()
print('names of features: ', list(phidf))

print('-------------------')
print('full data loaded')
print('-------------------')

Dataset size (6552, 23)
row number:  6552
feature number:  23

names of features:  ['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']
-------------------
full data loaded
-------------------


In [40]:
phidf=phidf.rename(columns={'Unnamed: 0': 'words'})
phidf.head(10)

,words,0,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,21
0,отличительный,5.398719e-08,1.441396e-06,4.467464e-05,0.000237,1.216818e-06,4.346776e-06,1.337085e-07,2.287369e-08,1.906509e-06,...,1.444734e-06,7.444236e-07,2.676352e-10,6.455061e-05,1.801329e-05,9.859061e-11,3.221635e-06,5.961753e-07,2.163929e-05,1.997642e-04
1,глаз,8.607737e-09,1.283897e-08,7.827887e-07,0.000298,2.540435e-04,5.391348e-07,3.075765e-08,2.050106e-08,7.628695e-07,...,1.457746e-07,6.309312e-08,6.325195e-10,1.188098e-06,2.291191e-06,6.504812e-09,1.391378e-07,8.825755e-06,2.796680e-04,2.258934e-03
2,старт,2.522970e-08,1.171134e-09,1.370788e-06,0.000029,2.619860e-06,1.180012e-06,1.656494e-07,2.383113e-07,2.150664e-06,...,4.764071e-05,1.682253e-06,3.615564e-05,8.853192e-06,4.685177e-03,2.007149e-05,1.924262e-06,7.062691e-10,4.698870e-07,6.371995e-09
3,подбирать,2.410562e-09,1.008187e-06,3.456765e-06,0.000255,7.348022e-04,9.601516e-07,3.118083e-07,7.321179e-08,3.495637e-07,...,7.046628e-05,1.172254e-05,1.631236e-05,5.248841e-04,2.590991e-05,1.947371e-12,3.083267e-05,4.636705e-06,3.131636e-05,4.037905e-04
4,развитие,2.131471e-07,5.255264e-08,9.790847e-06,0.002311,3.961898e-07,9.936725e-06,3.757119e-05,9.529764e-03,7.402505e-06,...,1.701174e-05,3.476317e-03,2.283559e-09,1.180044e-05,6.405407e-08,1.421116e-07,6.946084e-05,5.403399e-09,8.341891e-05,6.390103e-06
5,компенсировать,2.987843e-08,4.920225e-08,1.045854e-04,0.000020,5.466872e-05,4.685441e-06,7.012648e-06,2.071692e-05,9.588642e-07,...,6.532780e-05,1.452504e-06,2.019270e-07,3.375836e-06,5.934387e-05,2.076235e-11,3.558958e-04,1.107040e-06,7.985409e-06,5.073195e-04
6,ефим,2.469203e-07,7.315515e-07,1.028939e-05,0.000027,1.007897e-05,4.577691e-05,6.007561e-08,3.432048e-06,6.992034e-06,...,8.808604e-06,1.687420e-06,2.363112e-09,8.443147e-07,2.825568e-04,3.815915e-04,2.014433e-06,6.697040e-06,4.289459e-06,2.281629e-08
7,deutsch,6.347604e-10,8.358229e-11,1.815384e-04,0.000035,1.412193e-08,1.426826e-11,3.655778e-10,2.956976e-05,6.759703e-06,...,5.659544e-08,4.184852e-09,6.006616e-08,2.665913e-06,5.317739e-10,6.498022e-11,1.457275e-05,4.358655e-07,1.174183e-11,5.226106e-04
8,помощь,1.705136e-03,1.255621e-06,8.263650e-08,0.000033,4.888945e-06,2.879270e-05,2.653032e-07,3.018486e-06,3.605899e-03,...,3.746042e-05,4.260606e-06,5.340705e-09,1.623813e-06,2.224996e-06,4.898232e-08,1.044083e-05,9.700246e-07,1.570650e-06,3.080740e-02
9,представление,2.990764e-07,7.864500e-06,1.359897e-06,0.000191,8.940826e-07,5.059385e-06,4.634277e-04,5.910033e-06,3.732871e-04,...,4.391857e-07,3.628476e-05,1.274573e-07,1.430959e-04,7.924466e-06,2.558036e-08,1.781304e-07,3.264954e-04,1.510339e-07,6.463831e-06


In [41]:
# организуем сортировку по каждой колонке, 
# и будем сохранять слова в колонке 'words'
mytopics = []
# создаем список мен столбцов
topics_name =list(phidf)
topics_name.remove('words')
# цикл по именам столбцов
for i in topics_name:
    # сортируем по текущему столбцу
    sorted_df = phidf.sort_values(by=i, ascending = False)
    mytopics.append(sorted_df['words'].head(5).to_list())

In [10]:
print(mytopics)

[['организация', 'террористический', 'россия', 'запрещать', 'группировка'], ['полиция', 'происходить', 'мужчина', 'летний', 'сотрудник'], ['компания', 'российский', 'процент', 'тысяча', 'миллиард'], ['доллар', 'рубль', 'россия', 'процент', 'банк'], ['проводить', 'тысяча', 'работа', 'страна', 'место'], ['россия', 'президент', 'российский', 'украина', 'владимир'], ['россия', 'страна', 'отношение', 'министр', 'санкция'], ['россия', 'чемпионат', 'команда', 'российский', 'матч'], ['ребёнок', 'город', 'летний', 'штат', 'женщина'], ['самолёт', 'ракета', 'военный', 'борт', 'километр'], ['россия', 'закон', 'правительство', 'президент', 'глава'], ['работа', 'проводить', 'тысяча', 'страна', 'место'], ['учёный', 'исследование', 'журнал', 'результат', 'обнаруживать'], ['президент', 'партия', 'выбор', 'кандидат', 'пост'], ['проводить', 'работа', 'тысяча', 'страна', 'место'], ['компания', 'фильм', 'основывать', 'выпускать', 'представлять'], ['сеть', 'пользователь', 'видео', 'соцсеть', 'страница'], ['

In [44]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(topics=mytopics, corpus=gensim_corpus, dictionary=gensim_dictionary, coherence='u_mass')
coherence = cm.get_coherence()  # get coherence value
print(coherence)

KeyError: 'самолёт'